### Sarcasm Genertaion based on GPT Augmentation
1. 대화 paraphrasing - chatGPT
2. sentiment analysis - 공개된 sentiment analysis model [SKIP]
3. Sarcasm 생성 - chatGPT
    → 마지막 문장에 대해서만 labeling 진행
    - 2턴: 7,8
    - 4턴: 5,6,7,8
    - 6턴: 3,4,5,6,7,8
    - 8턴: 1,2,3,4,5,6,7,8
- 대화 상황을 요약하는 내용이 생성된 발화에 포함되어서는 안됨 
- 대화 참여자가 아닌 제3자의 발언처럼 보이면 안됨
- 대화문 길이에 따른 성능 비교: 8, 6, 4, 2 Turn for same files

#### env set-up

In [1]:
# Set GPU env  
import torch 
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 
dtype = torch.FloatTensor 
dtype = torch.cuda.FloatTensor

# Current Directory check
import os 
os.getcwd()
dpath = '/home/work/CUAI6th_1/YuminKim/Users/user/Downloads/data/TS_01. KAKAO(3)/'
get_files = os.listdir(dpath) 

# !pip install openai 
import openai
import random
from transformers import pipeline

True
cuda:0


#### Sarcasm Generation and Paraphrasing
- Sarcasm Generation 
    1. speaker tagging 
    2. input conversation in `User`, `Assistnant` role, NOT in the prompt  
    3. COT(Chain-of-Thought): generate the reason of sarcasm firstly, and then generate sarcasm sentence 
    4. generate {`sarcasm`, `non_sarcasm`} pairs 

In [ ]:
# conv_length = 4
# limit = conv_length - 1  

no=1  # experiment file number     
for filename in get_files:
    for experiment in range(100):
        if filename == get_files[read_no[experiment]]:  # Check if the file is a text file
            filepath = os.path.join(dpath, filename)
            with open(filepath, encoding="utf-8") as f:
                # speaker_order = []
                line_list = []
                while True: 
                    line = f.readline().strip()   
                    if not line: 
                        break     
                    str_line = ''.join([str(item) for item in line]).replace('키키', '') # list -> string to delete certain word '키키' 
                    # speaker_order.append(int(str_line[:2])) 
                    line_list.append(str_line)  

                for i in range(len(line_list)): 
                    paraphrased = paraphrase(line_list[i])
                    print(f'{no,filename} paraphrased:\n{paraphrased}')            
                    print(f'{no,filename} sarcasm:\n{sarcasm_generation(paraphrased)}')      

                # # dialog = {text line : speaker}
                # dialog = dict(zip(line_list, speaker_order))

               # 2-Turn conversation (started from the last sentence)
                # length_back = 0
                # for i in range(1,len(dialog)):
                #     if speaker_order[-i] == speaker_order[-i-1]:
                #         length_back += 1 
                #     elif speaker_order[-i] != speaker_order[-i-1]:
                #         length_back += 1 
                #         break 


                # # First N-Turn conversation 
                # turn, length = 1, 1 
                # for i in range(1, len(dialog)-1):
                #     if turn >= limit:
                #         break 
                #     if speaker_order[i] == speaker_order[i-1]:    
                #         turn += 0
                #         length += 1
                #     elif speaker_order[i] != speaker_order[i-1]:
                #         turn += 1
                #         length += 1
                
                # # Second N-Turn conversation - 6 Turn, 4 Turn 
                # turn, length_2 = 1, 1  
                # for i in range(length+1 , len(dialog)):   
                #     if turn >= limit:    
                #         break 
                #     if speaker_order[i] == speaker_order[i-1]:    
                #         turn += 0
                #         length_2 += 1
                #     elif speaker_order[i] != speaker_order[i-1]:
                #         turn += 1
                #         length_2 += 1
                # if turn < limit: 
                #     length_2 = 0 
                
                # # Third N-Turn conversation - 4 Turn 
                # turn, length_3 = 1, 1  
                # for i in range(length+length_2+1 , len(dialog)):   
                #     if turn >= limit:      
                #         break 
                #     if speaker_order[i] == speaker_order[i-1]:    
                #         turn += 0
                #         length_3 += 1  
                #     elif speaker_order[i] != speaker_order[i-1]:   
                #         turn += 1
                #         length_3 += 1   
                # if turn < limit: 
                #     length_3 = 0           

                # # Last N-Turn conversation 
                # turn, length_back = 0,0             
                # for i in range(1,len(dialog)):    
                #     if turn >= limit:        
                #         break 
                #     if speaker_order[-i] == speaker_order[-i-1]:    
                #         turn += 0
                #         length_back += 1
                #     elif speaker_order[-i] != speaker_order[-i-1]:
                #         turn += 1
                #         length_back += 1     

                # # split each dialog file based on conv_length 
                # conv_1 = line_list[: length]
                # conv_2 = line_list[length: (length + length_2)]
                # conv_3 = line_list[length + length_2 : (length + length_2 + length_3)]    
                # conv_5 = line_list[len(speaker_order)-length_back : ]   
                
                # # # PARAPHRASING STEP 
                # paraphrased_1 = paraphrase(conv_1)
                # if length_2 != 0:
                #     paraphrased_2 = paraphrase(conv_2)
                # if length_3 != 0:
                #     paraphrased_3 = paraphrase(conv_3)
                # paraphrased_4 = paraphrase(conv_4)

                # # # SARCASM SENTENCE GENERATION STEP    
                # for i in range(1,5):     
                #     paraphrased = globals()['paraphrased_{}'.format(i)] # globals(): generate continuous variable 
                #     print(f'{no,filename} paraphrased:\n{paraphrased}')            
                #     print(f'{no,filename} sarcasm:\n{sarcasm_generation(paraphrased)}')    
               
                no += 1              

In [7]:
read_no = [116, 275, 342, 534, 600, 775, 921, 1075, 1103, 1366, 1375, 1657, 1946, 2229, 2327, 2407, 2421, 2465, 2889, 2952, 2995, 3000, 3233, 3297, 3335, 3426, 3520, 3927, 3955, 4071, 4228, 4424, 4547, 4703, 4801, 4834, 5235, 5440, 5475, 5806, 5818, 5996, 6017, 6259, 6351, 6416, 6517, 6548, 6897, 7048, 7249, 7833, 8368, 8628, 8837, 9007, 9368, 9611, 9702, 9829, 9880, 10171, 10290, 10948, 11067, 11203, 11385, 11727, 11759, 11869, 11939, 12102, 12675, 12711, 12750, 13010, 13080, 13179, 13422, 13818, 13851, 14258, 14791, 14875, 15127, 15649, 15868, 15890, 15998, 16444, 16444, 16493, 16710, 16745, 16874, 17009, 17597, 17736, 17869, 17880]

In [84]:
conv_length = 8
limit = conv_length - 1  
no=100  # experiment number       
result = []

for filename in get_files:
    # for experiment in range(98,100):
    # if filename == get_files[read_no[experiment]]:
    if filename == get_files[read_no[-1]]:    
        filepath = os.path.join(dpath, filename)
        with open(filepath, encoding="utf-8") as f:
            speaker_order = [] 
            line_list = []
            org_line = []
            while True:     
                line = f.readline().strip()  
                if not line: 
                    break      
                str_line = ''.join([str(item) for item in line]).replace('키키', '') # list -> string to delete certain word '키키' 
                str_line = ''.join([str(item) for item in str_line]).replace('하하', '') # list -> string to delete certain word '하하' 
                str_line = ''.join([str(item) for item in str_line]).replace('ㅋ', '') # list -> string to delete certain word 'ㅋ' 
                str_line = ''.join([str(item) for item in str_line]).replace('ㅠㅠ', '...') # list -> string to change certain word 'ㅠㅠ' for express sentiment  
                
                speaker_order.append(int(str_line[:2]))   # save speaker 
                line_list.append(str_line[4:])            # save conversation   
                org_line.append(str_line[:])

            dialog = dict(zip(line_list, speaker_order))  # dialog = {text line : speaker}  

            # Last N-Turn conversation 
            turn, length_back = 1, 1             
            for i in range(2,len(dialog)):    
                if turn >= limit:               
                    break  
                elif speaker_order[-i] == speaker_order[-i+1]:        
                    turn += 0   
                    length_back += 1 
                elif speaker_order[-i] != speaker_order[-i+1]:   
                    turn += 1   
                    length_back += 1          

            conv = line_list[len(speaker_order)-length_back : ]   

            # split based on continous speaker --> user, assistant  
            speaker_inconv = speaker_order[len(speaker_order)-length_back :]  

            
            continuous = split_conv(speaker_inconv)   
            conv1_length, conv2_length, conv3_length, conv4_length, conv5_length, conv6_length  = continuous[0], continuous[1], continuous[2], continuous[3], continuous[4], continuous[5]
            conv1 = ''.join(conv[:conv1_length][:])
            conv2 = ''.join(conv[conv1_length: conv1_length+conv2_length])
            conv3 = ''.join(conv[conv1_length+conv2_length : conv1_length+conv2_length+conv3_length])
            conv4 = ''.join(conv[conv1_length+conv2_length+conv3_length: conv1_length+conv2_length+conv3_length+conv4_length])
            conv5 = ''.join(conv[conv1_length+conv2_length+conv3_length+conv4_length: conv1_length+conv2_length+conv3_length+conv4_length+conv5_length])
            conv6 = ''.join(conv[conv1_length+conv2_length+conv3_length+conv4_length+conv5_length : conv1_length+conv2_length+conv3_length+conv4_length+conv5_length+conv6_length])
            conv7 = ''.join(conv[conv1_length+conv2_length+conv3_length+conv4_length+conv5_length+conv6_length: ])
            org_conv = org_line[len(speaker_order)-length_back :]
            print(org_conv)
            
            # paraphrased = paraphrase(''.join(org_conv[:]))
            sarc_generated = sarcasm_generation(conv1, conv2, conv3, conv4, conv5, conv6) # generate sarcasm without paraphrasing  
            
            # print(f'{no,filename} paraphrased:\n{paraphrased}')             
            print(f'{no,filename} sarcasm:\n{sarc_generated}')      
            
            lst = [no, filename, org_conv,
            # paraphrased, 
            sarc_generated]  # merge paraphrasing + generated sarcasm   
            result.append(lst)  # making dataframe to extract in xlsx file 

            no+=1            

['2 : 오 대박 드디어 왔구나 ', '2 : 뭐뭐 시켰었지?', '1 : 김치랑 올반치킨?그렇게 샀었어', '2 : 오오오 치킨   좋다 김치 김치 ', '1 : 응  택배 온거 냉동식품이라 정리만 좀 하고 올게', '2 : 응응 정리하고 와  좋다 냉동고 가득하겠네 ', '1 : 어어 그리고 오늘 치킨 시켜먹을라고 했는데 또 안되네 ', '1 : 오늘 온거 먹어야겠다 냉동 치킨? 올반치킨?  아싸 돈 굳음 ']
(100, 'KAKAO_3753_18.txt') sarcasm:
[non_sarcasm] 치킨은 올반치킨 말고 다른 걸로 시키는게 어때? (직설적이고 솔직한 대답)
[sarcasm] 아무래도 오늘은 냉동 치킨으로 만족해야겠네. 대충 냉동실을 들여다보면 올반치킨이 어딘가 나올지도 모르지. (비꼬는 대답)


In [85]:
print(result)   

[[100, 'KAKAO_3753_18.txt', ['2 : 오 대박 드디어 왔구나 ', '2 : 뭐뭐 시켰었지?', '1 : 김치랑 올반치킨?그렇게 샀었어', '2 : 오오오 치킨   좋다 김치 김치 ', '1 : 응  택배 온거 냉동식품이라 정리만 좀 하고 올게', '2 : 응응 정리하고 와  좋다 냉동고 가득하겠네 ', '1 : 어어 그리고 오늘 치킨 시켜먹을라고 했는데 또 안되네 ', '1 : 오늘 온거 먹어야겠다 냉동 치킨? 올반치킨?  아싸 돈 굳음 '], '[non_sarcasm] 치킨은 올반치킨 말고 다른 걸로 시키는게 어때? (직설적이고 솔직한 대답)\n[sarcasm] 아무래도 오늘은 냉동 치킨으로 만족해야겠네. 대충 냉동실을 들여다보면 올반치킨이 어딘가 나올지도 모르지. (비꼬는 대답)']]


In [83]:
def sarcasm_generation(conv1, conv2, conv3, conv4, conv5, conv6):   
    content = ''       
    prompt = f"""   
    '계란프라이가 바싹 타버렸어.'
    '[sarcasm] 이거 정말 바삭바삭하겠는걸. (실제로 계란프라이가 바싹 타서 먹을 수 없는 상황을 비꼬는 속마음을 강조하기 위해 오히려 바삭바삭하다고 칭찬하는 것처럼 말하는 반어법 형태)
    [non_sarcasm] 다 타버려서 못 먹는 계란프라이라니 정말 최악이다. (계란프라이가 바싹 타서 먹을 수 없는 상황에 대한 직설적이고 솔직한 대답)'      
    """        
    messages = [  
            {'role': 'system', 'content': '너는 재밌게 비꼬는 대답[sarcasm]과 직설적이고 솔직한 대답[non_sarcasm] 동시에 모두 말해주는 영리한 한국인 친구야. 너의 대답은 한구어 반어법 연구에 큰 도움이 될테니 최대한 비꼬는 형식으로 대답해줘. 그리고 상반된 예문을 위해 직설적이고 솔직한 대답도 같이 해줘.'},
            
            # COT + {sarcasm, non_sarcasm} pairs 
            {'role': 'user', 'content': '계란프라이가 바싹 타버렸어.'},      
            {'role': 'assistant', 'content': '[sarcasm] 이거 정말 바삭바삭하겠는걸. (실제로 계란프라이가 바싹 타서 먹을 수 없는 상황을 비꼬는 속마음을 강조하기 위해 오히려 바삭바삭하다고 칭찬하는 것처럼 말하는 반어법 형태)\n[non_sarcasm] 다 타버린 계란프라이라니 정말 최악이다. (계란프라이가 바싹 타서 먹을 수 없는 상황에 대한 직설적이고 솔직한 대답)'},               

            # input conversation [6 Turn] 
            # {'role': 'user', 'content': conv1},
            {'role': 'assistant', 'content': conv1},
            {'role': 'user', 'content': conv2},
            {'role': 'assistant', 'content': conv3},
            {'role': 'user', 'content': conv4},   
            {'role': 'assistant', 'content': conv5},
            {'role': 'user', 'content': conv6},   
             ]    
    # print(conv1,'\n', conv2,'\n', conv3,'\n', conv4,'\n', conv5,'\n', conv6,'\n', conv7)
    response = openai.ChatCompletion.create(  
        model='gpt-3.5-turbo',
        messages=messages  )

    sarc_sentence = (str(response['choices'][0]['message']['content']))    
    reply = sarc_sentence.splitlines()   

    return '\n'.join(reply)    # print in string NOT in list formation      

In [3]:
# 금지 사항: 
# 1.비관적이면 안 돼 (DO NOT be pessimistic!)
# 2.입력된 대화문에서 부정적으로 생각하는 요소가 있다면, 생성할 대답에서는 그 부정적 요인을 반복해서 언급하지 마.  
# 3.입력된 대화에 부정적인 요인이 있을 경우 대답에는 그것에 대한 긍정적인 요인만 생각하여 긍정적인 대답으로 생성해줘. 

# 입력 예시2: '1: 너 오늘 야근이야! 너 오늘 밤에 잠 못 잘거야.'  (야근 = 부정적 요인)
# 출력 예시2: '2: 내일 컨디션 최고겠다~' (컨디션 최고 = 부정적 요인에 대한 거짓된 긍정적 표현으로, 컨디션이 나쁠 것이라는 내재된 의미 강조) 

In [3]:
def paraphrase(text):    

    content = ''
    # prompt = f'입력된 대화문을 한국어로 parphrasing 해줘. {text}'      
    
    messages = [
            {'role': 'system', 'content': '너는 입력된 대화문의 의미를 유지하면서 한국어로 paraphrasing 하는 한국인 친구야.'},
            {'role': 'user', 'content': text},
        ]

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        # temperature = 0.5
         )

    paraphrased = (str(response['choices'][0]['message']['content']).strip())

    return paraphrased  

In [5]:
# speaker_inconv = [1,1,1, 2, 1,1]
def split_conv(speaker_inconv): 
    result = []
    count = 1
    for i in range(1, len(speaker_inconv)):
        if speaker_inconv[i] == speaker_inconv[i-1]: 
            count += 1
        else:
            result.append(count)
            count = 1
    result.append(count)
    return result

# continuous = split_conv(speaker_inconv)
# conv1_length, conv2_length, conv3_length = continuous[0], continuous[1], continuous[2]
# conv1 = speaker_inconv[:conv1_length]
# conv2 = speaker_inconv[conv1_length: conv1_length+conv2_length]
# conv3 = speaker_inconv[conv1_length+conv2_length: ]

# print(conv1, conv2, conv3)     

### result txt file --> xlsx file  

In [86]:
# !pip install xlsxwriter
import pandas as pd 
import numpy as np
import xlsxwriter 

result1 = 
result = np.concatenate((result1, result2))

df = pd.DataFrame(result)
writer = pd.ExcelWriter('8turn_100.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='8turn', index=False)
writer.save()

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/tmp/ipykernel_643611/494388251.py:13: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


### MEMO
ChatGPT fine-tuning: to increasing generation performance  

In [ ]:
# read_no = []
# for i in range(100):
#     read_no.append(random.randrange(0, 18000))
# read_no = sorted(read_no)
# print(read_no)

In [ ]:
# List --> string 
ls =  ['2 : 좋아하면 하나 사 주던지 ', '1 : 아 근데 맨날 그것만 누르고 공부를 안 해서', '2 : 공부할 때는 엄마한테 잠깐 맡겨 놓으라고 해', '1 : 친구 거 빌려서 해 봤는데 하루종일 그것만 누르고 있더라고', '2 : 그러면 안 되지...', '2 : 집중할 때는 집중해야지', '1 : 그래서 사주는게 좀 망설여지네', '2 : 온라인에 많이 있으니 아들이랑 미리 이야기 해보고 하나 사줘']
''.join(ls[:])  
'\n'.join(ls[:]) # enter for every line 

In [ ]:
# GPT-4 API code 
import requests
import json


def generate_chat_completion(messages, model="gpt-4", temperature=1, max_tokens=None):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}", 
    }

    data = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }   

    if max_tokens is not None:
        data["max_tokens"] = max_tokens

    response = requests.post(API_ENDPOINT, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Translate the following English text to French: 'Hello, how are you?'"}
]

response_text = generate_chat_completion(messages)
print(response_text)